In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Conv2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


# 3개의 타입스탭으로 rnn 으로 학습하여
# 이전 3달의 승객수가 461,360,432 인경우 다음달 승객수 예측값을 구하시요. 

# 이후 다음 2달의 승객수도 예측하시오

def split_xy1(dataset, time_steps):
    x, y = list(), list()
    for i in range(len(dataset)):
        end_number = i + time_steps
        if end_number > len(dataset) -1:
            break
        tmp_x, tmp_y = dataset[i:end_number], dataset[end_number]
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)


dfAir = pd.read_csv('./international-airline-passengers.txt', index_col='time')

# 정규화
scale = MinMaxScaler()
dfAir = scale.fit_transform( dfAir['passengers'].values.reshape(-1,1) )

# 데이터 전처리 ( 인풋 x, 라벨 y값 정의 )
x_train, y_label = split_xy1(dfAir, 3)
#x = x_train.reshape(-1, 3, 1)

# 모델 정의
model = Sequential([
    SimpleRNN(64, activation='relu', input_shape=(3, 1)),
    Dense( units=32),
    Dense( units=1)
])

# 모델 최적화 정의
model.compile( loss='mse', optimizer='adam')

# 모델 학습
h = model.fit(x_train, y_label, epochs=500)

test_input_data = scale.transform(np.array( [461, 360, 432] ).reshape(3,1))

output = model.predict(test_input_data.reshape(1,3,1))
# 역정규화
output = scale.inverse_transform( output )
print("output value: ", output)
# plt.xticks(rotation=90)
# plt.plot( dfAir.loc['1960-01':])
# plt


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Conv2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


# 3개의 타입스탭으로 rnn 으로 학습하여
# 이전 3달의 승객수가 461,360,432 인경우 다음 2달 승객수 예측값을 구하시요. 

def split_xy2(dataset, time_steps, y_column):
    x, y = list(), list()
    for i in range(len(dataset)):
        x_end_number = i + time_steps
        y_end_number = x_end_number + y_column # 추가
        # if end_number > len(dataset) -1:
        # break
        if y_end_number > len(dataset): # 수정
            break
        tmp_x = dataset[i : x_end_number]
        tmp_y = dataset[x_end_number : y_end_number] # 수정
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)


dfAir = pd.read_csv('./international-airline-passengers.txt', index_col='time')


scaleF = MinMaxScaler()
dfAirN = scaleF.fit_transform( dfAir['passengers'].values.reshape(-1,1)  )
x,y = split_xy2( dfAirN.reshape(-1) ,3, 2 )

model = Sequential()
model.add( SimpleRNN( 32, activation='relu', input_shape=(3,1) ) )
model.add( Dense( units=32) )
model.add( Dense( units=2 ) )
model.compile( loss='mse', optimizer='adam')
h = model.fit( x.reshape(-1,3,1), y, epochs=1000 )

xn = scaleF.transform( np.array( [[461,360,432]] ).reshape(3,1) )

pred = model.predict( xn.reshape(1,3,1) )

scaleF.inverse_transform( pred )
